In [1]:
import sqlite3
import csv

In [2]:
file_csv = 'output_30.csv'

db_names = ['sqlite_db/books.sqlite',
            'sqlite_db/disney.sqlite',
            'sqlite_db/genes.sqlite',
            'sqlite_db/movie_platform.sqlite',
            'sqlite_db/shipping.sqlite']

In [3]:
def sqlite_execute(db_name, sql_query):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        # Convert rows to a string representation
        result = "; ".join([str(row) for row in rows])
    except Exception as e:
        result = f"Error: {e}"
    finally:
        conn.close()
    return result

In [4]:
# Function to update the CSV with SQL_op and LLM_SQL_op
def update_sql_outputs_in_csv(output_file, db_files):
    
    db_map = {
    'books': 'sqlite_db/books.sqlite',
    'disney': 'sqlite_db/disney.sqlite',
    'genes': 'sqlite_db/genes.sqlite',
    'movie_platform': 'sqlite_db/movie_platform.sqlite',
    'shipping': 'sqlite_db/shipping.sqlite'
    }
    
    updated_rows = []
    with open(output_file, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        
        if 'SQL_op' not in fieldnames:
            fieldnames.append('SQL_op')
        if 'LLM_SQL_op' not in fieldnames:
            fieldnames.append('LLM_SQL_op')
        
        for row in reader:
            db_id = row['db_id']
            sql_query = row['SQL']
            llm_sql_query = row.get('LLM_SQL', '')

            # Retrieve the corresponding database file from the map
            db_file = db_map.get(db_id)

            if db_file:
                row['SQL_op'] = sqlite_execute(db_file, sql_query)
                if llm_sql_query:
                    row['LLM_SQL_op'] = sqlite_execute(db_file, llm_sql_query)
            
            updated_rows.append(row)

    with open(output_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(updated_rows)


In [5]:
update_sql_outputs_in_csv(file_csv, db_files=db_names)